<a href="https://colab.research.google.com/github/jesus1080/CompuacionEvolutiva-/blob/master/Horarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deap

In [ ]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from google.colab import drive
import random
import numpy
import numpy as np
import pandas as pd


#Proyecto construccion de horarios de clase usando algoritmos evolutivos
#se tienen en cuenta 101 materias y 16 salones disponibles entre laboratorios y salones de teoria
#las materias de teoria pueden tener de 4 a 6 horas a la semana y los laboratorios 2 horas a la semana



In [ ]:
drive.mount('/content/drive')
# Ruta al archivo Excel en Google Drive
excel_file_path = '/content/drive/MyDrive/evolutivos/PIS_ASIGNATURAS.xlsx'
# archivo Excel en un DataFrame de pandas
df = pd.read_excel(excel_file_path)
df.index = range(1, len(df) + 1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#leer materias, profesores y salones

#crear individuo
def initIndividual(icls):

    # Crear la lista de materias con nombre y grupo según las repeticiones
    lista_materias = [[materia, grupo] for materia, grupo, horario in zip(df['NOMBRE'], df['GRUPO'], df['HORARIO']) for _ in range(horario//2)]


    # Obtener las posiciones de coincidencia en el DataFrame
    # Crear el vector de posiciones
    vector_posiciones = []
    for materia_grupo in lista_materias:
        posiciones = np.where((df['NOMBRE'] == materia_grupo[0]) & (df['GRUPO'] == materia_grupo[1]))[0]
        vector_posiciones.extend(posiciones + 1)

    dias = 5
    franjas = 5
    salones = 16
    horario = np.zeros((dias,franjas,salones))

    vector_horario = np.ravel(horario)

    # Generar índices aleatorios
    indices_aleatorios = np.random.choice(len(vector_horario), len(vector_posiciones), replace=False)
    vector_horario[indices_aleatorios] = vector_posiciones
    horario_reshape = np.reshape(vector_horario, (dias, franjas, salones))

    return icls(horario_reshape)

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, typecode="I", fitness=creator.FitnessMin, strategy=None)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
#verifica que en cada franja no se repitan materias del mismo semestre
def semfranja(ind):
    semestres = []
    sum = 0
    for dia in ind:
      for franja in dia:
        semestres = []
        for materia in franja:
          if materia != 0:
            semestre = df.loc[materia, 'SEMESTRE']
            semestres.append(semestre)
        conjunto = set(semestres)
        if len(conjunto) == len(semestres):
          sum += 1
        else:
          sum += 0
    return sum

In [ ]:
#verifica que en cada franja no se repita el profesor
def profesorenfranja(ind):
    profesores = []
    sum = 0

    for dia in ind:
      for franja in dia:
        profesores = []
        for materia in franja:
          if materia != 0:
            profesor = df.loc[materia, 'PROFESOR']
            profesores.append(profesor)
        conjunto = set(profesores)
        if len(conjunto) == len(profesores):
          sum += 1
        else:
          sum += 0
    return sum

In [ ]:
def evalOneMax(ind):
    sum = 0
    num = np.setdiff1d(np.arange(1, 102), ind)
    #penalisa si al horario le faltan materias
    if(num.shape[0] > 0):
      sum += -10
    for materia in np.nditer(ind):
      if materia != 0:
          tipo = df.loc[materia, 'TIPO']
          repsem = np.count_nonzero(ind == materia)
          coordenadas = np.argwhere(ind == materia)
          if tipo == 'L':
            if coordenadas[0][2] > 11 and repsem == 1:
              sum += 1
          else:
            if coordenadas[0][2] <= 11 and coordenadas[0][2] >= 0 and repsem == 2:
              sum += 1

    sum += semfranja(ind)
    sum += profesorenfranja(ind)

    return sum,

In [ ]:
def ajusteCruce(horarioHijo):
  num = np.setdiff1d(np.arange(1, 102), horarioHijo)
  indices = np.argwhere(horarioHijo == 0)
  if num.shape[0] > 0 :
    for i in num:
      rand = np.random.choice(indices.shape[0])
      indi = indices[rand]
      horarioHijo[indi[0]][indi[1]][indi[2]] = i
      indices = np.delete(indices, rand, axis=0)

  vector_horario = horarioHijo.reshape(-1)

  for i, row in df.iterrows():
      numero = i
      numero = np.int64(numero)
      apariciones = row['HORARIO']
      apariciones = apariciones // 2

      indices = np.where(vector_horario == numero)[0]

      if len(indices) > apariciones:
          excedentes = len(indices) - apariciones
          excedentes_indices = np.random.choice(indices, size=excedentes, replace=False)
          vector_horario[excedentes_indices] = 0

      elif len(indices) < apariciones:
          faltantes = apariciones - len(indices)
          disponibles_indices = np.where(vector_horario == 0)[0]

          if len(disponibles_indices) < faltantes:
              disponibles_indices = np.where(vector_horario != numero)[0]

          faltantes_indices = np.random.choice(disponibles_indices, size=faltantes, replace=False)
          vector_horario[faltantes_indices] = numero

  vector_horario = vector_horario.reshape(5,5,16)
  return vector_horario





In [ ]:
def cxTwoPointCopy(icls,ind1,ind2):
  cab1 = ind1[:3]
  col1 = ind1[3:5]
  cab2 = ind2[:3]
  col2 = ind2[3:5]

  h1 = np.concatenate((cab1, col2))
  h2 = np.concatenate((cab2, col1))

  h1 = icls(ajusteCruce(h1))
  h2 = icls(ajusteCruce(h2))

  return h1, h2

In [ ]:
def mutar(icls,ind):

  ind = np.array(ind)
  vec = ind.ravel()
  num = 35
  tam = 399
  indices = np.array([])
  while num != 0:
    index1 = random.randint(0, tam)
    index2 = random.randint(0, tam)
    indices = np.append(indices, index1)
    indices = np.append(indices, index2)
    while index1 == index2 and index1 in indices and index2 in indices:
        index1 = random.randint(0, tam)
        index2 = random.randint(0, tam)
    tem = np.copy(vec[index1])
    vec[index1] = vec[index2]
    vec[index2] = tem
    num += -1

  ind = vec.reshape(5,5,16)
  ind = icls(ajusteCruce(ind))

  return ind,

In [ ]:
toolbox = base.Toolbox()
toolbox.register("individual", initIndividual,creator.Individual )
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy,creator.Individual)
toolbox.register("mutate", mutar,creator.Individual)
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
def main():
    random.seed(64)

    pop = toolbox.population(n=300)

    # Numpy equality function (operators.eq) between two arrays returns the
    # equality element wise, which raises an exception in the if similar()
    # check of the hall of fame. Using a different equality function like
    # numpy.array_equal or numpy.allclose solve this issue.
    hof = tools.HallOfFame(1,similar=numpy.array_equal)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=100, stats=stats,
                      halloffame=hof)

    return pop, stats, hof

In [ ]:
if __name__ == "__main__":
    p,s,h=main()
    print(h[0])

gen	nevals	avg    	std   	min	max
0  	300   	117.477	7.4451	96 	134
1  	197   	122.85 	5.83502	101	141
2  	194   	125.707	5.34982	105	138
3  	204   	128.153	5.21375	114	143
4  	189   	130.2  	5.21089	116	143
5  	211   	130.957	5.93308	107	147
6  	179   	133.187	5.45513	117	149
7  	205   	134.387	6.06222	115	149
8  	209   	136.063	5.91039	120	149
9  	207   	137.477	6.03292	117	152
10 	201   	138.7  	6.20833	113	153
11 	176   	140.367	5.88038	122	153
12 	195   	141.63 	5.22492	126	152
13 	198   	141.94 	5.44332	123	157
14 	194   	142.643	5.70638	125	157
15 	186   	143.603	5.61243	121	157
16 	191   	144.73 	5.38985	123	157
17 	191   	145.377	5.49073	130	157
18 	215   	145.503	5.54406	127	157
19 	187   	146.473	5.77546	129	157
20 	201   	147.163	5.61338	130	159
21 	176   	147.863	5.30515	127	159
22 	197   	148.54 	5.35117	132	159
23 	195   	148.977	5.57639	123	159
24 	213   	149.52 	5.2386 	133	160
25 	206   	149.33 	5.70097	128	160
26 	180   	150.633	5.65086	133	161
27 	190   	151.763	5.4